In [1]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
from numpy.linalg import norm
from collections import Counter, defaultdict
from scipy.sparse import csr_matrix
from scipy import sparse
from sklearn.metrics.pairwise import cosine_distances
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import silhouette_score
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib as matplotlib
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import PCA
from sklearn.neighbors import DistanceMetric
from sklearn import preprocessing

In [2]:
# read in the dataset
df = pd.read_csv(
    filepath_or_buffer='../cleaning/example.csv', 
    index_col=0)

int_rate = df.iloc[:,19:20]

df=df.drop(['int_rate'], axis=1)

In [3]:
int_rate = int_rate['int_rate'].str.split('%').str.get(0)
int_rate

0           9.75
4          11.99
5          15.31
7          19.53
9          13.67
11          9.75
12         16.29
13         11.99
14          9.75
18          8.39
19         13.67
21         18.25
23         12.99
25         11.47
26         11.99
27         11.47
30          8.39
32          9.16
33         11.47
35         14.46
36         21.18
37         19.53
39         10.75
40         12.99
42          5.32
45         11.99
46         20.75
48         11.99
52         11.47
53         11.99
           ...  
1048163     6.19
1048164    19.42
1048183    15.49
1048186    10.07
1048207     6.83
1048225     9.43
1048227    14.52
1048242    14.52
1048274     7.46
1048276     9.58
1048286     7.46
1048296    11.05
1048301     6.83
1048334     7.46
1048353    12.13
1048382    14.52
1048395    16.46
1048403     8.08
1048417    10.07
1048436    15.04
1048501    10.56
1048510    10.90
1048518    18.45
1048527     8.08
1048547     6.19
1048548     9.58
1048549    13.58
1048560     5.

In [4]:
le = preprocessing.LabelEncoder()
df['amount_diff_inv'] = le.fit_transform(df['amount_diff_inv'])
df['grade'] = le.fit_transform(df['grade'])
df['home_ownership'] = le.fit_transform(df['home_ownership'])
df['verification_status'] = le.fit_transform(df['verification_status'])
df['purpose'] = le.fit_transform(df['purpose'])
df['loan_status'] = le.fit_transform(df['loan_status'])
df['initial_list_status'] = le.fit_transform(df['initial_list_status'])
df['delinq_2yrs_cat'] = le.fit_transform(df['delinq_2yrs_cat'])
df['inq_last_6mths_cat'] = le.fit_transform(df['inq_last_6mths_cat'])
df['pub_rec_cat'] = le.fit_transform(df['pub_rec_cat'])
df

,loan_amnt,amount_diff_inv,term,installment,grade,emp_length,home_ownership,annual_inc,verification_status,purpose,dti,delinq_2yrs_cat,inq_last_6mths_cat,open_acc,pub_rec,pub_rec_cat,acc_ratio,initial_list_status,loan_status,target
0,8400,1,36,270.06,1,2.0,1,66000.0,0,2,13.84,0,1,7,0,0,0.318182,1,2,0
4,20000,1,36,664.20,2,10.0,1,49000.0,1,2,15.58,1,0,8,0,0,0.285714,1,2,0
5,13625,1,60,326.36,2,1.0,1,50000.0,0,1,30.41,0,0,21,0,0,0.291667,1,2,0
7,15000,1,60,393.50,3,1.0,4,63890.0,0,2,20.36,0,1,7,0,0,0.875000,1,0,1
9,12000,1,60,277.18,2,10.0,1,68000.0,1,1,17.17,0,1,12,0,0,0.363636,1,4,0
11,5000,1,36,160.75,1,1.0,3,40000.0,1,2,10.38,0,1,25,1,1,0.555556,1,2,0
12,5000,1,36,176.51,3,10.0,1,105000.0,0,2,9.07,0,0,13,0,0,0.866667,1,0,1
13,4000,1,36,132.84,2,10.0,4,36000.0,2,2,16.40,0,0,5,0,0,0.714286,1,0,1
14,24000,1,36,771.60,1,5.0,1,50000.0,2,2,24.46,0,0,8,0,0,0.571429,1,0,1
18,18000,1,36,567.30,1,10.0,3,38200.0,0,1,24.07,1,0,19,0,0,0.463415,1,2,0


In [5]:
scaler = preprocessing.StandardScaler()
X_scaled = scaler.fit_transform(df)

In [6]:
X_scaled.shape

(392508, 20)

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, int_rate, test_size=0.33, random_state=21)


In [8]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf
from tensorflow import keras

import numpy as np

print(tf.__version__)

1.12.0


In [9]:
def build_model():
  model = keras.Sequential([
    keras.layers.Dense(64, activation=tf.nn.relu,
                       input_shape=(X_train.shape[1],)),
    keras.layers.Dense(64, activation=tf.nn.relu),
    keras.layers.Dense(1)
  ])

  optimizer = tf.train.RMSPropOptimizer(0.001)

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae'])
  return model

model = build_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                1344      
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 5,569
Trainable params: 5,569
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Display training progress by printing a single dot for each completed epoch
class PrintDot(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs):
    if epoch % 100 == 0: print('')
    print('.', end='')

EPOCHS = 500

# Store training stats
history = model.fit(X_train, y_train, epochs=EPOCHS,
                    validation_split=0.2, verbose=0,
                    callbacks=[PrintDot()])